In [31]:
import warnings
warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np

In [7]:
def GoldenCrossverSignal(name):
    path = f'../Data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date')
    data['Prev_Close'] = data.Close.shift(1)
    data['20_SMA'] = data.Prev_Close.rolling(window=20, min_periods=1).mean()
    data['50_SMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['Signal'] = 0
    data['Signal'] = np.where(data['20_SMA'] > data['50_SMA'], 1, 0)
    data['Position'] = data.Signal.diff()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    return df_pos

In [9]:
data = GoldenCrossverSignal('RELIANCE')

In [21]:
required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [18]:
data[data['Position'] == 'Sell'].index[-1]

Timestamp('2022-07-01 00:00:00')

In [20]:
# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price
class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        self.backtesting = self.backtesting.append(self.trade_log, ignore_index=True)
        

In [32]:
bt = Backtest()
capital = 50000
for index, data in required_df.iterrows():
    if(data.Position == 'Buy'):
        qty = capital // data.Open
        bt.buy('RELIANCE', index, data.Open, qty)
    else:
        bt.sell(index, data.Open, 'Exit Trigger', 0)

In [37]:
bt.backtesting

,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL,Exit Type,Holding Period
0,RELIANCE,Long Closed,1996-02-16,17.64,1996-04-05,16.14,2834.0,49988.304,-4251.00,-8.504,Exit Trigger,49 days
1,RELIANCE,Long Closed,1996-04-24,19.12,1996-06-18,18.64,2615.0,49989.425,-1255.20,-2.511,Exit Trigger,55 days
2,RELIANCE,Long Closed,1996-08-13,16.05,1996-08-16,15.98,3114.0,49988.536,-217.98,-0.436,Exit Trigger,3 days
3,RELIANCE,Long Closed,1996-08-30,16.33,1996-09-17,14.93,3061.0,49990.246,-4285.40,-8.572,Exit Trigger,18 days
4,RELIANCE,Long Closed,1996-11-12,14.19,1996-11-21,14.12,3523.0,49999.314,-246.61,-0.493,Exit Trigger,9 days
...,...,...,...,...,...,...,...,...,...,...,...,...
81,RELIANCE,Long Closed,2021-06-03,2221.00,2021-07-23,2134.45,22.0,48862.000,-1904.10,-3.897,Exit Trigger,50 days
82,RELIANCE,Long Closed,2021-08-27,2237.00,2021-11-24,2380.00,22.0,49214.000,3146.00,6.392,Exit Trigger,89 days
83,RELIANCE,Long Closed,2022-01-19,2530.00,2022-02-15,2351.20,19.0,48070.000,-3397.20,-7.067,Exit Trigger,27 days
84,RELIANCE,Long Closed,2022-03-29,2638.00,2022-05-27,2593.05,18.0,47484.000,-809.10,-1.704,Exit Trigger,59 days


In [38]:
bt.backtesting.to_csv('RELIANCE_TEST.csv')